# UNIVERSIDAD POLITÉCNICA SALESIANA 
# El VECINO - CUENCA

**Estudiante:** Gustavo Guallpa

**Profesor:** Ing. Diego Quisi

**Asignatura:** Simulación.

**Fecha:** 08/02/2020.

**Tema:** Examen Final de Simulación.

# IMPORTAMOS LA LIBRERIAS NECESARIAS.

In [47]:
import smtplib, sys, re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import tweepy
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import random
import json
from datetime import datetime, timedelta
from threading import Thread
from time import sleep
import csv

# RECUPERAMOS LOS DATOS CLIENTES QUE VAMOS A ENVIAR LOS CORREOS.
#Documento:https://docs.google.com/spreadsheets/d/1UvIRwmo7VTfRFmXlMkdxIWcI-ilb6wQh9964OhE_Zts/edit?usp=sharing
Se puede visualizar los nombres, apellidos , correo electrónico y su estado.

In [48]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("Clientes").sheet1

# Extract and print all of the values
list_of_hashes = sheet.col_values(3)


# MOSTRAMOS LOS CORREOS.

In [49]:
print(list_of_hashes)

['gguallpa97@outlook.es', 'frankling1997@hotmail.com', 'pedrogualllpa1234@gmail.com', 'ismaelcastillo.1996.960@gmail.com\r']


# MOSTRAMOS TODO LOS DATOS.

In [50]:
# Devuelve un array con los destinatarios en diccionario: 
#destinatario['nombre'] y destinatario['email']
def get_detinatarios(textfile):
    cont=0
    destinatarios = []
    # El fichero de donde lee los correos y nombres
    #print(lines)
    for line in textfile:
        #print(line)
        destinatario = {}
        #print(line_words)
        destinatario['nombre'] = textfile[cont][0]
        destinatario['apellido'] = textfile[cont][1]
        destinatario['email'] = textfile[cont][2]
        destinatario['estado'] = textfile[cont][3]
        destinatarios.append(destinatario)
        cont= cont+1
    return destinatarios

destinatarios = get_detinatarios(sheet.get_all_values())
print(destinatarios)

[{'nombre': 'Gustavo', 'apellido': 'Guallpa', 'email': 'gguallpa97@outlook.es', 'estado': 'Activo'}, {'nombre': 'Franklin', 'apellido': 'Guallpa', 'email': 'frankling1997@hotmail.com', 'estado': 'Activo'}, {'nombre': 'Pedro', 'apellido': 'Guallpa', 'email': 'pedrogualllpa1234@gmail.com', 'estado': 'Activo'}, {'nombre': 'Ismael', 'apellido': 'Castillo', 'email': 'ismaelcastillo.1996.960@gmail.com\r', 'estado': 'Activo'}]


# UTILIZANDO SELENIUM

In [51]:
consumer_key = "uLbkQhYPrTC9m9OFu9nPDNLqW"
consumer_secret = "KEavvCLW21schVttJiHM82ZfD2xHQwFnzJXNg2en1R65hIEdXx"
access_token = "773341738194862081-n6XhkmDWMQsSR8bMhWgWLnVo82y3I69"
access_token_secret = "y3HfYdEDNc63jrCmjfFaEK3jLMIDl0D4Z1M7Zb6rdIins"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


#Instancio el objeto.
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [56]:
url = 'https://twitter.com/login'

# change this path to where the chromedriver is on your machine
CHROMEDRIVERPATH = 'C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe'

# change this to your username and password
userUsername = ''
userPassword = ''

#GENERAMOS UN NUMERO RANDOMICO PARA ENVIAR EL FLAYER DE CUAQUIER CANDIDATO
def genCandidato():
    for i in range(1):
        candidato=random.randrange(1, 4, 1)
    return candidato

def login():
    browser = webdriver.Chrome(CHROMEDRIVERPATH)
    browser.get(url)

    browser.implicitly_wait(5)

    user = browser.find_element_by_name('session[username_or_email]')
    user.send_keys(userUsername)

    password = browser.find_element_by_name('session[password]')
    password.send_keys(userPassword)
    password.send_keys(Keys.ENTER)
    browser.implicitly_wait(5)
    
    candidat=0
    cuerpo=''
    candidato = genCandidato()
    if candidato == 1:
        cuerpo = '#Elecciones#2021 VOTA POR TU ASAMBLEISTA FAVORITA: DORIS SOLIZ'
        candidat=0
        print(cuerpo)
    if candidato == 2:
        cuerpo = '#Elecciones#2021 VOTA POR TU ASAMBLEISTA FAVORITO: ESTEBAN ALBORNOZ'
        candidat=1
        print(cuerpo)
    if candidato == 3:
        cuerpo = '#Elecciones#2021 VOTA POR TU ASAMBLEISTA FAVORITA: LOURDES CUESTA'
        candidat=2
        print(cuerpo)
    print(candidat)
    # ******************** ESCRIBIMOS QUE VAMOS A POSTEAR ********************
    tweet = browser.find_element_by_css_selector("br[data-text='true']")
    tweet.send_keys(cuerpo) 
    button = browser.find_element_by_css_selector("div[data-testid='tweetButtonInline']")
    button.click()
    
    
    
    # ******************** SUBIMOS LA IMAGEN ********************
    candidatos= ['dorissoliz.jpg','ealbornozv.jpg','LourdesCuestaO.jpg']
    media = api.media_upload(candidatos[candidat])

    # Post tweet with image
    tweet = cuerpo
    post_result = api.update_status(status=tweet, media_ids=[media.media_id])



    # ******************** ENVIAMOS EL CORREO ********************
    remitente = 'ASAMBLEISTAS AZUAY'
    destinatarios = list_of_hashes
    asunto = 'ELECCIONES-FEBRERO-2021'
    ruta_adjunto = candidatos[candidat]
    nombre_adjunto = candidatos[candidat]


    # Creamos el objeto mensaje
    mensaje = MIMEMultipart()

    # Establecemos los atributos del mensaje
    mensaje['From'] = remitente
    mensaje['To'] = ", ".join(destinatarios)
    mensaje['Subject'] = asunto

    # Agregamos el cuerpo del mensaje como objeto MIME de tipo texto
    mensaje.attach(MIMEText(cuerpo, 'plain'))

    # Abrimos el archivo que vamos a adjuntar
    archivo_adjunto = open(ruta_adjunto, 'rb')

    # Creamos un objeto MIME base
    adjunto_MIME = MIMEBase('application', 'octet-stream')
    # Y le cargamos el archivo adjunto
    adjunto_MIME.set_payload((archivo_adjunto).read())
    # Codificamos el objeto en BASE64
    encoders.encode_base64(adjunto_MIME)
    # Agregamos una cabecera al objeto
    adjunto_MIME.add_header('Content-Disposition', "attachment; filename= %s" % nombre_adjunto)
    # Y finalmente lo agregamos al mensaje
    mensaje.attach(adjunto_MIME)

    # Creamos la conexión con el servidor
    sesion_smtp = smtplib.SMTP('smtp.gmail.com', 587)

    # Ciframos la conexión
    sesion_smtp.starttls()

    # Iniciamos sesión en el servidor
    sesion_smtp.login('','')

    # Convertimos el objeto mensaje a texto
    texto = mensaje.as_string()

    # Enviamos el mensaje
    sesion_smtp.sendmail(remitente, destinatarios, texto)

    # Cerramos la conexión
    sesion_smtp.quit()
    
    print('CORREO ENVIADOS EXITOSAMENTE.')
    #Cerramos el navegador
    browser.close()

In [58]:
login()

#Elecciones#2021 VOTA POR TU ASAMBLEISTA FAVORITA: LOURDES CUESTA
2
CORREO ENVIADOS EXITOSAMENTE.


# REPORTE

In [59]:
def reporte(): 
    # ******************** GENERAMOS EL REPORTE ********************
    me = api.me()
    #print (json.dumps(me._json,indent=2))
    #Seccion de campos a extraer
    #remove("candidatos.csv")# Descomentar si se va a elimna para cargar con nuevos datos.
    #Seccion de campos a extraer
    name, user,follower,text, menciones, likes, hashtags, share = '','', '' ,'','','','',''
    #Nombres dentro de csv
    rows  = [['Nombre', 'usuario','followers','contenido','menciones','hashtags','likes','veces compartido']]
    contador_filas =0

    data = api.get_user('gguallpa97')
    print('Datos de mi Perfil: '+ data._json['name'])
    diccionario =data._json['entities'] 
    lista=[]
    
    for link in diccionario :  #recorremos 
        valor = diccionario[link]
        for vrd_valor in valor:
            lista = [lista,valor[vrd_valor]]
    
    print('https://twitter.com/gguallpa97')
    print('Tweets del Perfil.')
    name, user, followers = data._json['name'],data._json['screen_name'],data._json['followers_count']
    print(followers)
    for tweet in tweepy.Cursor(api.user_timeline,tweet_mode = 'extend').items(100):
        diccionario =tweet._json['entities'] 
        hashtags = diccionario['hashtags']
        menciones = diccionario['user_mentions']
        i=0;
        j=0;
        #print('HASHTAGS:')
        users_mnc=''
        hastags=''
        for dia in hashtags:
            hst = dia
            for links in hst:
                i=i+1
                if i == 1:
                    hastags =hst[links]
                    #print(hastags)

        #print('USERS MECIONADOS:')
        for m in menciones:
            mnc = m
           #print(mnc)
            for m_user in mnc:
                j=j+1
                if j == 1:
                    users_mnc=mnc[m_user]
                    #print(mnc[m_user])
        text, menciones, likes,share=tweet._json['text'].encode("utf-8"),users_mnc,tweet._json['favorite_count'],tweet._json['retweet_count']       
        single_row=[name, user, followers, text, menciones,hastags, likes, share]
        
        rows.append(single_row)
        
    with open('reporte.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)

In [60]:
reporte()

Datos de mi Perfil: Gustavo Guallpa
https://twitter.com/gguallpa97
Tweets del Perfil.
10


In [46]:
class Temporizador(Thread):
    def __init__(self, hora, delay, funcion):
        # El constructor recibe como parámetros:
        ## hora = en un string con formato hh:mm:ss y es la hora a la que queremos que se ejecute la función.
        ## delay = tiempo de espera entre comprobaciones en segundos.
        ## funcion = función a ejecutar.

        super(Temporizador, self).__init__()
        self._estado = True
        self.hora = hora
        self.delay = delay
        self.funcion = funcion

    def stop(self):
        self._estado = False

    def run(self):
        # Pasamos el string a dato tipo datetime
        aux = datetime.strptime(self.hora, '%H:%M:%S')
        # Obtenemos la fecha y hora actuales.
        hora = datetime.now()
        # Sustituimos la hora por la hora a ejecutar la función.
        hora = hora.replace(hour = aux.hour, minute=aux.minute, second=aux.second, microsecond = 0)
        # Comprobamos si la hora ya a pasado o no, si ha pasado sumamos un dia (hoy ya no se ejecutará).
        if hora <= datetime.now():
            hora += timedelta(days=1)
        print('Ejecución automática iniciada')
        print('Proxima ejecución programada el {0} a las {1}'.format(hora.date(),  hora.time()))

        # Iniciamos el ciclo:
        while self._estado:
            # Comparamos la hora actual con la de ejecución y ejecutamos o no la función.
            ## Si se ejecuta sumamos un dia a la fecha objetivo.
            if hora <= datetime.now():
                self.funcion()
                print('Ejecución programada ejecutada el {0} a las {1}'.format(hora.date(),  hora.time()))
                hora += timedelta(days=1)
                login()
                reporte()
                print('Próxima ejecución programada el {0} a las {1}'.format(hora.date(),  hora.time()))

            # Esperamos x segundos para volver a ejecutar la comprobación.
            sleep(self.delay)

        #Si usamos el método stop() salimos del ciclo y el hilo terminará.
        else:
             print('Ejecución automática finalizada')


#=========================================================================================
#Ejemplo de uso:

def ejecutar():
    print('Función ejecutada')

t = Temporizador('05:56:00',1,ejecutar)# Instanciamos nuestra clase Temporizador
t.start() #Iniciamos el hilo

#Mientras el programa principal puede seguir funcinando:
#sleep(2)
for _ in range(10):
    print('A espera de la ejecución')
    sleep(2)

# Si en cualquier momento queremos detener el hilo desde la aplicacion simplemete usamos el método stop()
#sleep(120) # Simulamos un tiempo de espera durante el cual el programa principal puede seguir funcionando. 
#t.stop()   # Detenemos el hilo.

Ejecución automática iniciadaA espera de la ejecución

Proxima ejecución programada el 2021-02-08 a las 05:56:00
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
A espera de la ejecución
Función ejecutada
Ejecución programada ejecutada el 2021-02-08 a las 05:56:00
VOTA POR TU ASAMBLEISTA FAVORITO: 2
1
CORREO ENVIADOS EXITOSAMENTE.
Datos de mi Perfil: Gustavo Guallpa
https://twitter.com/gguallpa97
Tweets del Perfil.
9
Próxima ejecución programada el 2021-02-09 a las 05:56:00
